In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("benchmark.csv")
df

,variance,dp_variance,epsilon,beta,m,L,U,size,loc,scale,variant
0,1.139474e-08,1.227493e-07,0.01,0.001667,1,0.000826,0.001154,10,0.001,0.0001,normal
1,1.139474e-08,3.722363e-07,0.01,0.001667,1,0.000826,0.001154,10,0.001,0.0001,normal
2,1.139474e-08,3.701055e-08,0.01,0.001667,1,0.000826,0.001154,10,0.001,0.0001,normal
3,1.139474e-08,1.494259e-06,0.01,0.001667,1,0.000826,0.001154,10,0.001,0.0001,normal
4,1.139474e-08,1.401241e-05,0.01,0.001667,1,0.000826,0.001154,10,0.001,0.0001,normal
...,...,...,...,...,...,...,...,...,...,...,...
6075,2.007117e+00,5.492129e-01,10.00,1.666667,1000,-11.111866,7.566741,10000,0.000,1.0000,laplace
6076,2.007117e+00,5.453093e-01,10.00,1.666667,1000,-11.111866,7.566741,10000,0.000,1.0000,laplace
6077,2.007117e+00,5.594263e-01,10.00,1.666667,1000,-11.111866,7.566741,10000,0.000,1.0000,laplace
6078,2.007117e+00,5.482418e-01,10.00,1.666667,1000,-11.111866,7.566741,10000,0.000,1.0000,laplace


In [3]:
agged = df.groupby(["variant", "loc", "scale", "size", "epsilon"]).agg(
    variance=("variance", "first"),
    mean_dp_variance=("dp_variance", "mean"),
    CoV_dp_variance=("dp_variance", lambda x: np.std(x) / np.mean(x)),
    min=("dp_variance", "min"),
    q25=("dp_variance", lambda x: np.quantile(x, 0.25)),
    q50=("dp_variance", lambda x: np.quantile(x, 0.50)),
    q75=("dp_variance", lambda x: np.quantile(x, 0.75)),
    max=("dp_variance", "max"),
).reset_index()
agged = agged.rename(
    dict(
        variant="distribution",
        loc="$\mu$",
        scale="$\sigma$",
        size="$n$",
        epsilon="$\epsilon$",
        mean_dp_variance="mean of DP sample variance",
        CoV_dp_variance="CoV of DP sample variance",
        q25="25%",
        q50="50%",
        q75="75%",
    ),
    axis=1,
)
agged

,distribution,$\mu$,$\sigma$,$n$,$\epsilon$,variance,mean of DP sample variance,CoV of DP sample variance,min,25%,50%,75%,max
0,laplace,0.0,1.0,10,0.01,3.687549e+00,8.536145e+01,6.231516,-7.439901e+02,-2.089746e+02,-7.822229e+00,1.867539e+02,1.300670e+03
1,laplace,0.0,1.0,10,0.10,3.687549e+00,-1.533621e+01,-7.160957,-2.075534e+02,-5.291685e+01,-3.207378e+01,-1.146824e+01,1.779097e+02
2,laplace,0.0,1.0,10,1.00,3.687549e+00,-4.618261e-01,-13.912417,-1.715785e+01,-1.612170e+00,6.944948e-01,1.393340e+00,8.433223e+00
3,laplace,0.0,1.0,10,10.00,3.687549e+00,3.812542e+00,1.853168,-1.000662e+00,-3.487828e-01,4.693675e-01,2.435620e+00,1.930967e+01
4,laplace,0.0,1.0,100,0.01,2.320280e+00,-5.385798e+00,-26.308433,-2.760101e+02,-8.450700e+01,4.202731e+01,9.709190e+01,1.656906e+02
...,...,...,...,...,...,...,...,...,...,...,...,...,...
603,normal,1000000.0,10000000.0,1000,10.00,1.036716e+14,4.642168e+13,0.075566,4.457376e+13,4.484748e+13,4.526789e+13,4.557110e+13,5.680153e+13
604,normal,1000000.0,10000000.0,10000,0.01,9.936624e+13,4.741345e+13,0.469730,-9.384124e+12,4.136609e+13,4.667091e+13,6.088774e+13,7.393129e+13
605,normal,1000000.0,10000000.0,10000,0.10,9.936624e+13,4.208687e+13,0.065271,3.712484e+13,4.126256e+13,4.267694e+13,4.453050e+13,4.497018e+13
606,normal,1000000.0,10000000.0,10000,1.00,9.936624e+13,4.339643e+13,0.048880,4.105559e+13,4.274433e+13,4.298338e+13,4.307145e+13,4.952273e+13


In [4]:
table_1 = agged.to_markdown(index=False, tablefmt="github")
print(table_1)

| distribution   |    $\mu$ |    $\sigma$ |   $n$ |   $\epsilon$ |        variance |   mean of DP sample variance |   CoV of DP sample variance |               min |               25% |               50% |              75% |              max |
|----------------|----------|-------------|-------|--------------|-----------------|------------------------------|-----------------------------|-------------------|-------------------|-------------------|------------------|------------------|
| laplace        |    0     |      1      |    10 |         0.01 |     3.68755     |                 85.3614      |                 6.23152     |    -743.99        |    -208.975       |      -7.82223     |    186.754       |   1300.67        |
| laplace        |    0     |      1      |    10 |         0.1  |     3.68755     |                -15.3362      |                -7.16096     |    -207.553       |     -52.9169      |     -32.0738      |    -11.4682      |    177.91        |
| laplace        |    0 

In [5]:
df2 = df.copy()
df2["abs_dev"] = (df2["variance"] - df2["dp_variance"]).abs()
df2["rel_dev"] = df2["abs_dev"] / df2["variance"]
agged2 = df2.groupby(["size", "epsilon"]).agg(
    mean_rel_dev=("rel_dev", "mean"),
    std_rel_dev=("rel_dev", "std"),
).reset_index()
agged2 = agged2.rename(
    dict(
        size="$n$",
        epsilon="$\epsilon$",
        U="$U$",
        mean_rel_dev="mean rel. deviation",
        std_rel_dev="std of the rel. deviation",
    ),
    axis=1,
)
agged2

,$n$,$\epsilon$,mean rel. deviation,std of the rel. deviation
0,10,0.01,338.865201,1527.243368
1,10,0.10,29.552201,137.453713
2,10,1.00,3.746089,24.491365
3,10,10.00,0.687286,1.001965
4,100,0.01,137.374052,949.683152
5,100,0.10,312.458478,5917.668375
6,100,1.00,0.891666,1.210146
7,100,10.00,0.658247,0.358514
8,1000,0.01,14.168902,54.327308
9,1000,0.10,1.660069,4.787988


In [6]:
table_2 = agged2.to_markdown(index=False, tablefmt="github")
print(table_2)

|   $n$ |   $\epsilon$ |   mean rel. deviation |   std of the rel. deviation |
|-------|--------------|-----------------------|-----------------------------|
|    10 |         0.01 |            338.865    |                1527.24      |
|    10 |         0.1  |             29.5522   |                 137.454     |
|    10 |         1    |              3.74609  |                  24.4914    |
|    10 |        10    |              0.687286 |                   1.00197   |
|   100 |         0.01 |            137.374    |                 949.683     |
|   100 |         0.1  |            312.458    |                5917.67      |
|   100 |         1    |              0.891666 |                   1.21015   |
|   100 |        10    |              0.658247 |                   0.358514  |
|  1000 |         0.01 |             14.1689   |                  54.3273    |
|  1000 |         0.1  |              1.66007  |                   4.78799   |
|  1000 |         1    |              0.691916 |    